In [2]:
import pandas as pd

In [5]:
df = pd.read_csv('datasets/public_train.csv')

In [8]:
df.head()

,id,user_name,post_message,timestamp_post,num_like_post,num_comment_post,num_share_post,label
0,1,389c669730cb6c54314a46be785cea42,"THĂNG CẤP BẬC HÀM ĐỐI VỚI 2 CÁN BỘ, CHIẾN SỸ H...",1585945439,19477,378,173.0,0
1,2,775baa6d037b6d359b229a656eaeaf08,<URL>,1588939166.0,11,5,3,0
2,3,b9f3394d2aff86d85974f5040c401f08,TƯ VẤN MÙA THI: Cách nộp hồ sơ để trúng tuyển ...,1591405213,48,5,19.0,0
3,4,808e278b22ec6b96f2faf7447d10cd8e,Cơ quan Cạnh tranh và Thị trường Anh quyết địn...,1592023613,3,0,0.0,0
4,5,f81bdd6d8be4c5f64bb664214e47aced,Thêm 7 ca tại Quảng Nam liên quan đến hành khá...,1583737358,775,0,54.0,0


In [7]:
df.iloc[:,0][0]

1

In [14]:
for i in range(20):
    print(df.iloc[:,0][i])
    print('*'*100)

4373,a80e9faa68fdb250bcea6455851e5298,"Mai th?´n Ch?­ Trung á» HÆ°ng Y?ªn dá»¡ bá» phong tá»a sau 29 ng? y c?¡ch ly:
****************************************************************************************************
Mai th?´n Ch?­ Trung ÄÆ°á»£c dá»¡ phong tá»a, c?³ g?¬ báº¡n chia sáº» b? i gi?ºp nh?©! C?¡m Æ¡n báº¡n!
****************************************************************************************************
nan
****************************************************************************************************
,1588171905,9,0,
4374,03d27f8d9815f6eb8ac8cfdd9c07e283,Tá»± h? o Viá»t Nam l?ªn táº­n 3 s?³ng truyá»n h?¬nh ná»i tiáº¿ng cá»§a Má»¹ CNN ABC CBN :3 Táº¥t cáº£ má»i ngÆ°á»i c?¹ng cá» l?ªn, Äáº©y l?¹i Corona n? o! ",,59000,1100,8800
****************************************************************************************************
4375,8d8954f983a48be041a2236ffe8787e8,"UBND TP Cáº§n ThÆ¡ ra c?´ng vÄn cho há»c sinh, sinh vi?ªn, há»c vi?ªn tr?ªn Äá»a b? n 